In [1]:
# Import dependencies
import pandas as pd
import time

from pprint import pprint
from bs4 import BeautifulSoup
from splinter import Browser
from selenium import webdriver

# https://splinter.readthedocs.io/en/latest/api/driver-and-element-api.html

In [2]:
executable_path = {"executable_path": "chromedriver.exe"}
browser = Browser("chrome", **executable_path, headless=True)
time.sleep(1)

In [3]:
url = 'https://mars.nasa.gov/news/'
browser.visit(url)
time.sleep(3)

In [4]:
mars_html = browser.html
mars_soup = BeautifulSoup(mars_html, 'html.parser')
#t_f = browser.is_element_present_by_css("ul.item_list li.slide",wait_time=1)
#print(t_f)
time.sleep(3)

In [5]:
# Extract the first news title and paragraph text

try:
    news_title = mars_soup.find('ul', class_="item_list").find('li',class_="slide"). \
        find('div',class_="content_title").text
    print(f"news_title = {news_title}") 
    
    news_p = mars_soup.find('ul',class_="item_list").find('li',class_="slide"). \
        find('div',class_="article_teaser_body").text
    print(f"news_p = {news_p}")
    
except AttributeError as e:
    print(e)

news_title = What's Mars Solar Conjunction, and Why Does It Matter?
news_p = NASA spacecraft at Mars are going to be on their own for a few weeks when the Sun comes between Mars and Earth, interrupting communications.


In [6]:
url = 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'
browser.visit(url)
time.sleep(2)

In [7]:
# Find and click the Full Image button
full_image_button = browser.find_by_id('full_image')
full_image_button.click()
time.sleep(2)

In [8]:
# Find and click More Info button
more_info_button = browser.find_link_by_partial_text('more info')
more_info_button.click()
time.sleep(2)

In [9]:
# Get featured_image_url
mars_html = browser.html
mars_images_soup = BeautifulSoup(mars_html, 'html.parser')

try:    
    url = mars_images_soup.find('figure', class_='lede').a['href']
    #print(f"featured_image_url: {featured_image_url}") 
    
except AttributeError as e:
    print(e)
    
featured_image_url = "https://www.jpl.nasa.gov" + url
print(f"full url: {featured_image_url}")    

full url: https://www.jpl.nasa.gov/spaceimages/images/largesize/PIA16605_hires.jpg


In [10]:
# Get latest Mars Weather Report from Twitter
url = 'https://twitter.com/marswxreport?lang=en)'
browser.visit(url)
time.sleep(2)
mars_weather_html = browser.html
mars_weather_soup = BeautifulSoup(mars_weather_html, 'html.parser')
time.sleep(1)

# Scrape the latest tweet
mars_tweet = mars_weather_soup.find('p', class_='TweetTextSize').text
print(f"tweet: {mars_tweet}")

tweet: InSight sol 261 (2019-08-21) low -102.4ºC (-152.4ºF) high -26.6ºC (-15.8ºF)
winds from the SSE at 4.9 m/s (11.0 mph) gusting to 16.0 m/s (35.8 mph)
pressure at 7.70 hPapic.twitter.com/MhPPOHJg3m


In [11]:
# Scrape Space-Facts.com for Mars statistics
url = 'https://space-facts.com/mars/'
earth_mars_facts = pd.read_html(url)
mars_facts_df = earth_mars_facts[1]
mars_facts_df.columns = ['Mars Statistic','Value']
mars_facts_df

,Mars Statistic,Value
0,Equatorial Diameter:,"6,792 km"
1,Polar Diameter:,"6,752 km"
2,Mass:,6.39 × 10^23 kg (0.11 Earths)
3,Moons:,2 (Phobos & Deimos)
4,Orbit Distance:,"227,943,824 km (1.38 AU)"
5,Orbit Period:,687 days (1.9 years)
6,Surface Temperature:,-87 to -5 °C
7,First Record:,2nd millennium BC
8,Recorded By:,Egyptian astronomers


In [12]:
mars_facts_df.to_html('./templates/mars_facts.html', index=False )

In [13]:
# Scrape USGS Astrogeology site for Mars hemisphere images
url = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'
browser.visit(url)
html = browser.html
usgs_soup = BeautifulSoup(html, 'html.parser')

# Get list of hemisphere blocks
hemisphere_blocks = usgs_soup.find('div', class_='collapsible results'). \
    find_all('div',class_='item')

In [14]:
# Initialize list
hemisphere_images = []

# Loop through hemisphere results and append to list
for hemisphere in hemisphere_blocks:
    
    # Get title and cut off 'enhanced' from end
    title = hemisphere.find('div', class_='description'). \
        find('a', class_='itemLink product-item').h3.text
    print(title)
    if title.endswith(' Enhanced'):
        title = title[:-9]
    print(title)
  
    # Construct each hemisphere image URL
    url = 'https://astrogeology.usgs.gov'
    full_url = url + hemisphere.find('a',class_='itemLink product-item')['href']    
    browser.visit(full_url)
    time.sleep(3)
    
    usgs_html = browser.html
    image_soup = BeautifulSoup(usgs_html,'html.parser')
    image_url = image_soup.find('div',class_='downloads').a['href']
    
    hemisphere_dict = {
        "title" : title,
        "image_url" : image_url
    }

    # Construct list of hemisphere dictionaries  
    hemisphere_images.append(hemisphere_dict)

Cerberus Hemisphere Enhanced
Cerberus Hemisphere
Schiaparelli Hemisphere Enhanced
Schiaparelli Hemisphere
Syrtis Major Hemisphere Enhanced
Syrtis Major Hemisphere
Valles Marineris Hemisphere Enhanced
Valles Marineris Hemisphere


In [15]:
hemisphere_images

[{'title': 'Cerberus Hemisphere',
  'image_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg'},
 {'title': 'Schiaparelli Hemisphere',
  'image_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg'},
 {'title': 'Syrtis Major Hemisphere',
  'image_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif/full.jpg'},
 {'title': 'Valles Marineris Hemisphere',
  'image_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg'}]